In [10]:
pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install dgl.sparse 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement dgl.sparse (from versions: none)
ERROR: No matching distribution found for dgl.sparse


In [21]:
pip install dgl==0.9


     ---------------------------------------- 2.8/2.8 MB 22.4 kB/s eta 0:00:00
  Attempting uninstall: dgl
    Found existing installation: dgl 1.0.1
    Uninstalling dgl-1.0.1:
      Successfully uninstalled dgl-1.0.1


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\lenaa\\AppData\\Local\\Continuum\\miniconda3\\envs\\DL_segmentation\\Lib\\site-packages\\~gl\\dgl.dll'
Consider using the `--user` option or check the permissions.



In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import scipy.sparse as sp

import dgl
import numpy as np
import torch
import torch.nn.functional as F
from dgl.dataloading import GraphDataLoader
from sklearn.model_selection import train_test_split
from time import time
from tqdm import tqdm

from functions_HGPLS import train, test

In [2]:
dataset = dgl.data.FakeNewsDataset('gossipcop', 'bert')

In [3]:
dataset = dgl.data.GINDataset("PROTEINS", self_loop=False)

In [2]:
dataset, labels = dgl.load_graphs("../data/HIV_dgl_graphs")

In [3]:
labels = labels['glabel'].tolist()

In [4]:
dataset[5].ndata

{'atomic': tensor([[8.],
        [8.],
        [6.],
        [6.],
        [6.],
        [6.],
        [7.],
        [6.],
        [6.]])}

In [4]:
new_dataset = list()
for i in range(len(dataset)):
    #g, l = dataset[i]
    g = dataset[i]
    l = labels[i]
    #g.ndata["feature"] = torch.ones_like(dataset.feature[g.ndata["_ID"]])
    g.ndata["feature"] = g.ndata["atomic"]
    g = dgl.add_self_loop(g)
    g = dgl.add_reverse_edges(g)
    new_dataset.append((g, l))

In [5]:
train_dataset, test_dataset = train_test_split(new_dataset, test_size=0.25, random_state=42)

train_dataloader = GraphDataLoader(train_dataset, batch_size=16, drop_last=False)
test_dataloader = GraphDataLoader(test_dataset, batch_size=16, drop_last=False)

In [6]:
from dgl.nn.pytorch import GATConv

class GAT(nn.Module):
    def __init__(self, in_feats, h_feats, n_classes):
        super(GAT, self).__init__()
        self.layer1 = GATConv(in_feats, h_feats, num_heads=4)
        self.layer2 = GATConv(4*h_feats, h_feats, num_heads=4)
        self.layer3 = GATConv(4*h_feats, h_feats, num_heads=6)
        self.fc = nn.Linear(h_feats, n_classes)
        self.elu = nn.ELU()

    def forward(self, g, in_feat):
        x1 = self.layer1(g, in_feat)
        x1 = self.elu(x1)
        x1 = x1.view(in_feat.shape[0], -1)
        x2 = self.layer2(g, x1)
        x2 = self.elu(x2)
        x2 = x2.view(in_feat.shape[0], -1)
        x3 = self.layer3(g, x2)
        x3 = torch.mean(x3, dim=1)
        with g.local_scope():
            g.ndata['h'] = x3
            x4 = dgl.readout_nodes(g, 'h')
        return F.log_softmax(self.fc(x4), dim=-1)

In [7]:
# Load model architecture
device = 'cpu' if torch.cuda.is_available() else 'cpu'
model = GAT(in_feats=1, n_classes=3, h_feats=128).to(device)

In [25]:
# Define optimizer and loss
optimizer = torch.optim.Adam(
        model.parameters(), lr=0.00001, weight_decay=0.0001
    )
loss = torch.nn.CrossEntropyLoss()

In [26]:
# Train model and keep the best validation loss model
bad_cound = 0
best_val_acc = 0
best_epoch = 0
epochs = 10
patience = 10
print_every = 1
train_times = []
for e in range(epochs):
    s_time = time()
    train_loss, train_acc = train(model, optimizer, loss, train_dataloader, device)
    train_times.append(time() - s_time)
    val_acc, val_loss = test(model, loss, test_dataloader, device)
    if best_val_acc < val_acc:
        best_val_loss = val_loss
        bad_cound = 0
        best_epoch = e + 1
        torch.save(model.state_dict(), "models/GATModel_prot.pt")
    else:
        bad_cound += 1
    if bad_cound >= patience:
        break

    if (e + 1) % print_every == 0:
        log_format = (
            "Epoch {}: train_loss={:.4f}, train_acc={:.4f}, val_acc={:.4f}, vall_loss={:.4f}"
        )
        print(log_format.format(e + 1, train_loss, train_acc, val_acc, val_loss))
print(
    "Best Epoch {}, final test loss {:.4f}".format(
        best_epoch, best_val_loss
    )
)

  0%|          | 0/115 [00:00<?, ?it/s]c:\Users\lenaa\AppData\Local\Continuum\miniconda3\envs\DL_segmentation\lib\site-packages\dgl\backend\pytorch\tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \
100%|██████████| 39/39 [00:02<00:00, 17.72it/s]


Epoch 1: train_loss=1.0838, train_acc=0.4247, val_acc=0.4746, vall_loss=1.0541


100%|██████████| 39/39 [00:02<00:00, 14.18it/s]


Epoch 2: train_loss=1.0785, train_acc=0.4230, val_acc=0.4664, vall_loss=1.0539


100%|██████████| 39/39 [00:02<00:00, 16.32it/s]


...

Epoch 7: train_loss=1.0769, train_acc=0.4274, val_acc=0.4697, vall_loss=1.0533


100%|██████████| 39/39 [00:02<00:00, 15.65it/s]


Epoch 8: train_loss=1.0767, train_acc=0.4279, val_acc=0.4714, vall_loss=1.0531


 26%|██▌       | 30/115 [00:06<00:17,  4.81it/s]


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1837056 bytes.

In [20]:
pred=[]
lab = []
for batch in tqdm(test_dataloader):
        batch_graphs, batch_labels = batch
        batch_graphs = batch_graphs.to(device)
        batch_labels = batch_labels.long().to(device)
        out = model(batch_graphs, batch_graphs.ndata["feature"].to(dtype=torch.float32))
        pred += out.argmax(dim=1).tolist()
        lab += batch_labels.tolist()

  0%|          | 0/39 [00:00<?, ?it/s]c:\Users\lenaa\AppData\Local\Continuum\miniconda3\envs\DL_segmentation\lib\site-packages\dgl\backend\pytorch\tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \
100%|██████████| 39/39 [00:02<00:00, 13.17it/s]


In [22]:
import sklearn

In [24]:
sklearn.metrics.confusion_matrix(lab, pred)

array([[151, 105,   5],
       [110, 144,   1],
       [ 27,  68,   0]], dtype=int64)